V.0.0

IMPORTAMOS LIBRERIAS

In [1]:
import os
import pandas as pd
import openpyxl
from pathlib import Path
import re

INICIALIZAMOS DATAFRAME GLOBAL

In [49]:
df_compilado = pd.DataFrame()

FUNCION QUE PROCESA LOS CERTIFICADOS

In [50]:
import os
import pandas as pd
import openpyxl
from pathlib import Path
import re

# Definir la función para procesar un archivo Excel
def procesar_excel(file_path, sheet_name, data_certificados):
    # Leer el archivo Excel, especificando que los nombres de columnas están en la fila 14 (índice 13)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=14 )

    # Eliminar las primeras dos filas no deseadas
    df = df.drop([0, 1])

    # Restablecer el índice
    df.reset_index(drop=True, inplace=True)

    # Renombrar las columnas correctamente
    df.columns = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"] + df.columns[6:].tolist()

    # Transformar al formato largo
    id_vars = ["Método de Análisis", "Parámetro", "CM", "Unidad", "LD", "LQ"]
    melted_data = pd.melt(df, id_vars=id_vars, var_name='Sample_ID', value_name='Valor')
    melted_data['Ruta'] = file_path
    #Extraer el segmento de la ruta Proyecto
    path = Path(file_path)
    
    # Obtener las partes de la ruta
    path_parts = path.parts
    start_segment = "Certificados"
    # Encontrar el índice del segmento de inicio
    try:
        start_index = path_parts.index(start_segment) + 1
    except ValueError:
        return "Segmento no encontrado en la ruta."
    
    # Extraer el segmento de la ruta
    extracted_path = path_parts[start_index]
    
    # Usar una expresión regular para eliminar el número y el punto iniciales
    cleaned_path = re.sub(r'^\d+\.\s*', '', extracted_path)
    
    # Añadir la columna de Proyecto al DataFrame
    melted_data['Proyecto'] = cleaned_path
    
    #Obtener Campaña
    path = Path(file_path)
    
    # Obtener las partes de la ruta
    path_parts = path.parts
    start_segment = "Certificados"
    # Encontrar el índice del segmento de inicio
    try:
        start_ = path_parts.index(start_segment) + 2
    except ValueError:
        return "Segmento no encontrado en la ruta."
    
    # Extraer el segmento de la ruta
    extracted_campaña = path_parts[start_]
    
    # Usar una expresión regular para eliminar el número y el punto iniciales
    campaña = re.sub(r'^\d+\.\s*', ' ', extracted_campaña)
    #agregar campaña como columna
    melted_data['Campaña'] = campaña
    
    # Cargamos nuevamente los datos originales de la hoja de resultados
    original_data = pd.read_excel(file_path, sheet_name=sheet_name)

# # extraer las filas con los datos de identificación, fecha y tipo de muestra
    identification_data = original_data.iloc[9:14, 1:].transpose()
    identification_data.columns = identification_data.iloc[0]
    identification_data = identification_data.drop(identification_data.index[0])
    identification_data.reset_index(drop=True, inplace=True)

# # Renombrando las columnas para facilitar el acceso
    identification_data.columns = ["N° Informe_LB", "Fecha de Muestreo", "Hora de Muestreo", "Tipo de Muestra", "FLOW"]

# # Unimos los datos de identificación con los datos fundidos basándonos en el índice
    combined_data = pd.merge(melted_data, identification_data, left_on='Sample_ID', right_on='FLOW')

# # Eliminamos la columna 'Sample_ID' y ajustamos el dataframe para que 'FLOW' sea el nuevo 'Sample_ID'
    combined_data.drop('Sample_ID', axis=1, inplace=True)
    combined_data.rename(columns={'FLOW': 'Sample_ID'}, inplace=True)

# Reorganizando las columnas para que 'Sample_ID', 'Fecha de Muestreo', y 'Tipo de Muestra' aparezcan primero
    column_order = ['Sample_ID', 'Fecha de Muestreo', "Hora de Muestreo", 'Tipo de Muestra', 'Método de Análisis', 'Parámetro','Proyecto','Campaña','Ruta', 
                'CM', 'Unidad', 'LD', 'LQ', 'N° Informe_LB', 'Valor']
    combined_data = combined_data[column_order]

# # Mostramos los primeros registros para verificar que los cambios se han realizado correctamente
# combined_data.head()
    combined_data
    # LIMPIEZA DE DATA
    combined_data_cleaned = combined_data[combined_data['Parámetro'] != 'Fecha de Análisis']
   # Ajustar el formato de la columna "Fecha de Muestreo"
    try:
        combined_data_cleaned['Fecha de Muestreo'] = pd.to_datetime(combined_data_cleaned['Fecha de Muestreo'], format='%d/%m/%Y',errors='coerce')
    except Exception as e:
        print(f"Error ajustando el formato de la fecha: {e}")
    
    # <<<<<<<<<<<<<<<<<<<<<<    
    # Renombrar las columnas
    new_column_names = {
    'Sample_ID': 'SampleID',
    'Fecha de Muestreo': 'Fecha_de_Muestreo',
    'Hora de Muestreo': 'Hora_de_Muestreo',
    'Tipo de Muestra': 'Tipo_de_Muestra',
    'Método de Análisis': 'Metodo_de_Analisis',
    'Parámetro': 'Parametro',
    'Proyecto': 'Proyecto',
    'Campaña': 'Campana',
    'Ruta': 'Ruta_Archivo',
    'CM': 'CM',
    'Unidad': 'Unidad',
    'LD': 'LD',
    'LQ': 'LQ',
    'N° Informe_LB': 'ID_CERTIFICADO_LAB',
    'Valor': 'Valor'
    }
    combined_data_renamed = combined_data_cleaned.rename(columns=new_column_names)
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>
    
    # ------------------------------
    # Ajustar el formato de las siguientes columnas
    combined_data_renamed['Valor'] = combined_data_renamed['Valor'].astype(str).str.replace(',', '.')

    combined_data_renamed['Unidad'] = combined_data_renamed['Unidad'].astype(str).str.replace('---', '')

    combined_data_renamed['LD'] = combined_data_renamed['LD'].astype(str).str.replace('---', '')

    combined_data_renamed['LQ'] = combined_data_renamed['LQ'].astype(str).str.replace('---', '')
    # ------------------------------
    
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # Crear las columnas 'Qualifier' y 'Valor_BEST' a partir de la columna 'Valor'
    combined_data_renamed['Qualifier'] = combined_data_renamed['Valor'].apply(lambda x: '<' if '<' in str(x) else None)
    combined_data_renamed['Valor_BEST'] = combined_data_renamed['Valor'].apply(lambda x: float(str(x).replace('<', '').strip()) if '<' in str(x) else x)
    combined_data_renamed['Valor_BEST_LD'] = combined_data_renamed.apply(lambda row: row['Valor_BEST'] / 2 if row['Qualifier'] == '<' else row['Valor_BEST'], axis=1)
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    
    # ------------------------------
    # Cambia en columna parametros los espacios por guiones bajos
    combined_data_renamed['Parametro'] = combined_data_renamed['Parametro'].apply(lambda x: x.replace(' ', '_'))
    # ------------------------------
    
    # >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    # Establecer formato a las columnas
    combined_data_renamed['Valor_BEST'] = combined_data_renamed['Valor_BEST'].astype('float64')
    combined_data_renamed['Valor_BEST_LD'] = combined_data_renamed['Valor_BEST_LD'].astype('float64')
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    
    #  ------------------------------
    # Generar columna Parametro_Unidad
    combined_data_renamed['Parametro_Unidad'] = combined_data_renamed['Parametro'] + "_" + combined_data_renamed['Unidad']
    # ------------------------------
    
    # Añadir los datos procesados al DataFrame global
    data_certificados = pd.concat([data_certificados, combined_data_renamed], ignore_index=True)

    return data_certificados

# Definir el directorio y el nombre de la hoja


# Obtener todos los archivos Excel en el directorio





INGRESAR PARAMETROS REQUERIDOS POR LA FUNCION

In [59]:
directory_path = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\05. AMSA - EIA EVU\ALS"
sheet_name = "Resultados"
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Crear un DataFrame vacío para almacenar todos los datos
data_certificados = pd.DataFrame()

EJECUTAR FUNCION

In [60]:
# Procesar cada archivo Excel encontrado
for file in excel_files:
    file_path = os.path.join(directory_path, file)  # Construir la ruta completa al archivo
    try:
        print(f"Procesando el archivo {file}... de la ruta {file_path}...")
        print("total de archivos =", len(excel_files))
        data_certificados = procesar_excel(file_path, sheet_name, data_certificados)
    except Exception as e:
        print(f"Error procesando el archivo {file_path}: {e}")

# Mostrar las primeras filas del DataFrame combinado
data_certificados

Procesando el archivo 53097_2020_FLOW HIDRO.xlsx... de la ruta G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\Certificados\05. AMSA - EIA EVU\ALS\53097_2020_FLOW HIDRO.xlsx...
total de archivos = 1


C:\Users\fdona\AppData\Local\Temp\ipykernel_25460\3899360959.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data_cleaned['Fecha de Muestreo'] = pd.to_datetime(combined_data_cleaned['Fecha de Muestreo'], format='%d/%m/%Y',errors='coerce')


,SampleID,Fecha_de_Muestreo,Hora_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,Proyecto,Campana,Ruta_Archivo,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor,Qualifier,Valor_BEST,Valor_BEST_LD,Parametro_Unidad
0,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Conductividad Eléctrica (SM) INN/SMA,Conductividad_Eléctrica,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11046,µS/cm,2,200000,448128/2020-1.0,4324,None,4324.0000,4324.0000,Conductividad_Eléctrica_µS/cm
1,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Solidos Disueltos Totales (SM) INN/SMA,Sólidos_Disueltos_Totales,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11075,mg/L,10,,448128/2020-1.0,2980,None,2980.0000,2980.0000,Sólidos_Disueltos_Totales_mg/L
2,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,"pH, Agua (SM) INN/SMA",pH,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11102,,,,448128/2020-1.0,7.76,None,7.7600,7.7600,pH_
3,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,"pH, Agua (SM) INN/SMA",T°_de_pH,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11102,°C,,,448128/2020-1.0,20.9,None,20.9000,20.9000,T°_de_pH_°C
4,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Aniones por Cromatografía Iónica (EPA 300.1) I...,Fluoruro,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11551,mg/L,,"0,06",448128/2020-1.0,< 0.06,<,0.0600,0.0300,Fluoruro_mg/L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Metal Total ICP-MS (EPA) INN/SMA,Zinc_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,12680,mg/L,"0,0007","0,0007",448142/2020-1.0,0.0332,None,0.0332,0.0332,Zinc_Total_mg/L
416,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Total (SM 2320-A) INN/SMA,Alcalinidad_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14271,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Alcalinidad_Total_mg CaCO3/L
417,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Bicarbonato (SM 2320-A) INN/SMA,Bicarbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14272,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Bicarbonato_mg CaCO3/L
418,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Carbonato (SM 2320-A) INN,Carbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14275,mg CaCO3/L,1,,448142/2020-1.0,< 1,<,1.0000,0.5000,Carbonato_mg CaCO3/L


In [61]:
data_certificados.dtypes

SampleID                      object
Fecha_de_Muestreo     datetime64[ns]
Hora_de_Muestreo              object
Tipo_de_Muestra               object
Metodo_de_Analisis            object
Parametro                     object
Proyecto                      object
Campana                       object
Ruta_Archivo                  object
CM                            object
Unidad                        object
LD                            object
LQ                            object
ID_CERTIFICADO_LAB            object
Valor                         object
Qualifier                     object
Valor_BEST                   float64
Valor_BEST_LD                float64
Parametro_Unidad              object
dtype: object

In [62]:
df_compilado = pd.concat([df_compilado, data_certificados], ignore_index=True)


In [63]:
df_compilado

,SampleID,Fecha_de_Muestreo,Hora_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,Proyecto,Campana,Ruta_Archivo,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor,Qualifier,Valor_BEST,Valor_BEST_LD,Parametro_Unidad
0,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Conductividad Eléctrica (SM) INN/SMA,Conductividad_Eléctrica,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11046,µS/cm,2,200000,448128/2020-1.0,4324,None,4324.0000,4324.0000,Conductividad_Eléctrica_µS/cm
1,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Solidos Disueltos Totales (SM) INN/SMA,Sólidos_Disueltos_Totales,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11075,mg/L,10,,448128/2020-1.0,2980,None,2980.0000,2980.0000,Sólidos_Disueltos_Totales_mg/L
2,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,"pH, Agua (SM) INN/SMA",pH,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11102,,,,448128/2020-1.0,7.76,None,7.7600,7.7600,pH_
3,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,"pH, Agua (SM) INN/SMA",T°_de_pH,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11102,°C,,,448128/2020-1.0,20.9,None,20.9000,20.9000,T°_de_pH_°C
4,CS28-AS-4,2020-10-25,09:35:00,Agua Residual,Aniones por Cromatografía Iónica (EPA 300.1) I...,Fluoruro,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11551,mg/L,,"0,06",448128/2020-1.0,< 0.06,<,0.0600,0.0300,Fluoruro_mg/L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Metal Total ICP-MS (EPA) INN/SMA,Zinc_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,12680,mg/L,"0,0007","0,0007",448142/2020-1.0,0.0332,None,0.0332,0.0332,Zinc_Total_mg/L
416,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Total (SM 2320-A) INN/SMA,Alcalinidad_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14271,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Alcalinidad_Total_mg CaCO3/L
417,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Bicarbonato (SM 2320-A) INN/SMA,Bicarbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14272,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Bicarbonato_mg CaCO3/L
418,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Carbonato (SM 2320-A) INN,Carbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14275,mg CaCO3/L,1,,448142/2020-1.0,< 1,<,1.0000,0.5000,Carbonato_mg CaCO3/L


CONEXION A LA BASE DE DATOS

In [64]:
#cargar a la base de datos con sqlalchemy
import psycopg2
from sqlalchemy import create_engine

# Credenciales db
host = 'localhost'
database = 'certificados_dbo'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name = 'certificados_Flow'

df_data_bd = pd.read_sql_table(table_name, con=engine)#Trae la tabla de la bda un dataframe

df_data_bd


,SampleID,Fecha_de_Muestreo,Hora_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,Proyecto,Campana,Ruta_Archivo,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor,Qualifier,Valor_BEST,Valor_BEST_LD,Parametro_Unidad
0,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Carbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.00000,0.50000,Alcalinidad_Carbonato_mg CaCO3/L
1,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Bicarbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.00000,222.00000,Alcalinidad_Bicarbonato_mg CaCO3/L
2,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Total,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.00000,222.00000,Alcalinidad_Total_mg CaCO3/L
3,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Hidróxido,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.00000,0.50000,Alcalinidad_Hidróxido_mg CaCO3/L
4,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Conductividad Eléctrica (SM) INN/SMA,Conductividad_Eléctrica,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11046,µS/cm,2,200000,305656/2022-1.0,524,None,524.00000,524.00000,Conductividad_Eléctrica_µS/cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14337,FLOW-MLP-198,2024-01-31,07:45:00,Agua Subterránea,Metal Total ICPMS,Estaño_Total,Keypro,Nodo 5 y Mauro,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,26532,mg/L,"0,00010","0,00050",64627/2024-1.0,< 0.00050,<,0.00050,0.00025,Estaño_Total_mg/L
14338,FLOW-MLP-198,2024-01-31,07:45:00,Agua Subterránea,Metal Total ICPMS,Titanio_Total,Keypro,Nodo 5 y Mauro,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,26532,mg/L,"0,00010","0,00020",64627/2024-1.0,0.00050,None,0.00050,0.00050,Titanio_Total_mg/L
14339,FLOW-MLP-198,2024-01-31,07:45:00,Agua Subterránea,Metal Total ICPMS,Uranio_Total,Keypro,Nodo 5 y Mauro,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,26532,mg/L,"0,00001","0,00006",64627/2024-1.0,0.00111,None,0.00111,0.00111,Uranio_Total_mg/L
14340,FLOW-MLP-198,2024-01-31,07:45:00,Agua Subterránea,Metal Total ICPMS,Vanadio_Total,Keypro,Nodo 5 y Mauro,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,26532,mg/L,"0,00010","0,00030",64627/2024-1.0,0.01389,None,0.01389,0.01389,Vanadio_Total_mg/L


CONCATENAMOS DATA DE LA BD Y LA NUEVA DATA

In [65]:
df_update= pd.concat([df_data_bd, df_compilado], ignore_index=True)

In [66]:
df_update

,SampleID,Fecha_de_Muestreo,Hora_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,Proyecto,Campana,Ruta_Archivo,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor,Qualifier,Valor_BEST,Valor_BEST_LD,Parametro_Unidad
0,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Carbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.0000,0.5000,Alcalinidad_Carbonato_mg CaCO3/L
1,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Bicarbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.0000,222.0000,Alcalinidad_Bicarbonato_mg CaCO3/L
2,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Total,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.0000,222.0000,Alcalinidad_Total_mg CaCO3/L
3,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Hidróxido,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.0000,0.5000,Alcalinidad_Hidróxido_mg CaCO3/L
4,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Conductividad Eléctrica (SM) INN/SMA,Conductividad_Eléctrica,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11046,µS/cm,2,200000,305656/2022-1.0,524,None,524.0000,524.0000,Conductividad_Eléctrica_µS/cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14757,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Metal Total ICP-MS (EPA) INN/SMA,Zinc_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,12680,mg/L,"0,0007","0,0007",448142/2020-1.0,0.0332,None,0.0332,0.0332,Zinc_Total_mg/L
14758,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Total (SM 2320-A) INN/SMA,Alcalinidad_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14271,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Alcalinidad_Total_mg CaCO3/L
14759,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Bicarbonato (SM 2320-A) INN/SMA,Bicarbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14272,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Bicarbonato_mg CaCO3/L
14760,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Carbonato (SM 2320-A) INN,Carbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14275,mg CaCO3/L,1,,448142/2020-1.0,< 1,<,1.0000,0.5000,Carbonato_mg CaCO3/L


ENVIAMOS LA NUEVA DATA A LA BBDD

In [67]:
df_update.to_sql(table_name, con=engine, if_exists='replace', index=False)

762

TRAEMOS NUEVAMENTE DATA DE LA BD PARTA REVISION

In [68]:
df_data_bd = pd.read_sql_table(table_name, con=engine)#Trae la tabla de la bda un dataframe

df_data_bd

,SampleID,Fecha_de_Muestreo,Hora_de_Muestreo,Tipo_de_Muestra,Metodo_de_Analisis,Parametro,Proyecto,Campana,Ruta_Archivo,CM,Unidad,LD,LQ,ID_CERTIFICADO_LAB,Valor,Qualifier,Valor_BEST,Valor_BEST_LD,Parametro_Unidad
0,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Carbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.0000,0.5000,Alcalinidad_Carbonato_mg CaCO3/L
1,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Bicarbonato,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.0000,222.0000,Alcalinidad_Bicarbonato_mg CaCO3/L
2,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Total,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,222,None,222.0000,222.0000,Alcalinidad_Total_mg CaCO3/L
3,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Alcalinidades (SM 2320-A) INN/SMA,Alcalinidad_Hidróxido,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11030,mg CaCO3/L,1,,305656/2022-1.0,< 1,<,1.0000,0.5000,Alcalinidad_Hidróxido_mg CaCO3/L
4,FLOW-MLP-12,2022-06-02,14:32:00,Agua Subterránea,Conductividad Eléctrica (SM) INN/SMA,Conductividad_Eléctrica,Portezuelo,00 Campaña mayo SGS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,11046,µS/cm,2,200000,305656/2022-1.0,524,None,524.0000,524.0000,Conductividad_Eléctrica_µS/cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14757,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Metal Total ICP-MS (EPA) INN/SMA,Zinc_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,12680,mg/L,"0,0007","0,0007",448142/2020-1.0,0.0332,None,0.0332,0.0332,Zinc_Total_mg/L
14758,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Total (SM 2320-A) INN/SMA,Alcalinidad_Total,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14271,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Alcalinidad_Total_mg CaCO3/L
14759,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Bicarbonato (SM 2320-A) INN/SMA,Bicarbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14272,mg CaCO3/L,1,,448142/2020-1.0,50,None,50.0000,50.0000,Bicarbonato_mg CaCO3/L
14760,CS28-AS-8,2020-10-25,12:04:00,Agua Residual,Alcalinidad Carbonato (SM 2320-A) INN,Carbonato,AMSA - EIA EVU,ALS,G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYu...,14275,mg CaCO3/L,1,,448142/2020-1.0,< 1,<,1.0000,0.5000,Carbonato_mg CaCO3/L


In [5]:
import pandas as pd


ruta = r"G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW\BDD_IsoTerreno_MLP-Flow_SoloCodUtilizados.xlsx"

df = pd.read_excel(ruta, sheet_name='Descripción muestra', header=4)

# df.rename(columns={'Unnamed: 9':'cm'}, inplace=True)

In [7]:
#eliminar columna unnamed: 31 y unnamed: 32 y 33
df.drop(['Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33'], axis=1, inplace=True)
# df.drop(['Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)





In [9]:
df

Index(['ID_Muestra', 'ID_Estacion', 'ESTACION', 'Consultor', 'Tramo_Desde',
       'Tramo_Hasta', 'FechaMuestra', 'Hora', 'ID_InternaLab',
       'ID_Interna_Iso', 'Informe_Qca', 'Informe_Iso', 'Tipo_Muestra',
       'Muestra QAQC', 'Campaña', 'Lab_Qca', 'Lab_Iso', 'Obs_Muestra',
       'Archivo_Muestra_HQCA', 'Ruta_archivo_Muestra_HQCA',
       'Fecha_ingreso_Muestra_HQCA', 'Ingreso_Muestra_HQCA_por',
       'Archivo_Muestra_Isotop', 'Ruta_archivo_Muestra_Isotop',
       'Fecha_ingreso_Muestra_Isotop', 'Ingreso_Muestra_Isotop_por',
       'Obs_muestreado', 'Quimica', 'Isotopo', 'Terreno', 'Muestreado',
       'Oxigeno-18_‰', 'Deuterio_‰', 'Azufre-34 en SO4_‰',
       'Oxigeno-18 en SO4_‰', 'Tritio_TU', '15N_NO3_‰', '18O_NO3_‰',
       'Bromuro-81_‰', 'Cloruro-37_‰', 'Estroncio-87/Estroncio-86_‰',
       'pH (Campo)_u de pH', 'pH mV (Campo)_mV', 'Potencial Redox (Campo)_mV',
       'Oxigeno Disuelto Porcentual (Campo)_%',
       'Oxígeno Disuelto (Campo)_mg/L', 'Conductividad (Campo)_u

In [10]:

import pandas as pd
import openpyxl
import os
ruta = r'G:\.shortcut-targets-by-id\1ugEph6zRyoKHbwpiYuXbB3Pj0UXueIKJ\09. Contrato CS115 EIA 2024\03.WIP\03.Hidroquímica\4. Muestras FLOW'
nombre_archivo = 'Extracto_Isoterreno.xlsx'

ruta_completa = os.path.join(ruta, nombre_archivo)
df.to_excel(ruta_completa, index=False)